<a href="https://colab.research.google.com/github/s-ahuja/gesture_recognition/blob/master/Conv%203D%20Model%20Execution%20-%20Google%20Collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Jun 12 08:44:34 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# Clone the entire repo.
data_folder = '/content/cloned-repo'
root_dir = f'{data_folder}/Project_data'

In [3]:
full_restart = False
if (full_restart):
  import shutil
  shutil.rmtree(data_folder,ignore_errors=True)
  import os
  os.makedirs(data_folder,exist_ok=True)
  os.chdir(data_folder)
  !git clone -l -s "https://github.com/s-ahuja/gesture_recognition.git" {data_folder}
%cd {data_folder}/Project_data
!ls

/content/cloned-repo/Project_data
03-model_init_2019-06-1207_24_37.204208  train.csv	     val.csv
model_init_2019-06-1208_43_38.397048	 train_images.pkl    val_images.pkl
numpy					 training.cnn3d.txt


In [0]:
import pickle
filehandler = open('train_images.pkl',"rb")
train_images = pickle.load(filehandler)
filehandler.close()
filehandler = open('val_images.pkl',"rb")
val_images = pickle.load(filehandler)
filehandler.close()

In [5]:
len(train_images[0]),len(val_images[0])

(19890, 3000)

In [0]:
import glob, shutil
model_directories = glob.glob(f'{root_dir}/model*')
for directory in model_directories:
    shutil.rmtree(directory)

# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [0]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import glob
from cv2 import imread,resize,cvtColor,COLOR_BGR2RGB,INTER_AREA
import datetime
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
# import sys
# old_stdout = sys.stdout
# log_file = open('logfile.txt', 'w')
# sys.stdout = log_file
# print('Starting Log at {}'.format(datetime.datetime.now()))
# #sys.stdout = old_stdout
# #log_file.close()

We set the random seed so that the results don't vary drastically.

In [8]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In [9]:
img_indices = list(range(1,30,2))
print(img_indices)
input_shape=(len(img_indices),100,100,3) #15 images per video, each image is of 100x100x3
print(input_shape)
batch_size = 30
no_of_classes = 5
num_epochs = 50 # choose the number of epochs

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29]
(15, 100, 100, 3)


In [10]:
train_doc = np.random.permutation(open(f'{root_dir}/train.csv').readlines())
val_doc = np.random.permutation(open(f'{root_dir}/val.csv').readlines())

# filter out records which are not considered during experiments
train_doc = [x for x in train_doc if x.split(';')[0] in [paths.split('/')[-2] for paths in train_images[0]]] 
val_doc = [x for x in val_doc if x.split(';')[0] in [paths.split('/')[-2] for paths in val_images[0]]]

print('no. of training video seq=',len(train_doc),'no. of validation video seq=',len(val_doc))

no. of training video seq= 663 no. of validation video seq= 100


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [0]:
def generator(source_path, folder_list, batch_size, ablation=False):
    #print('Source path = ', source_path, '; batch size =', batch_size)
    # if ablation is true then only take only 2 images else take much more.
    img_idx = img_indices #create a list of image numbers you want to use for a particular video. 
    x = len(img_idx)    
    y,z = input_shape[1],input_shape[2] # image_standard_size
    Model_Execution = True
    while Model_Execution:
        t = np.random.permutation(folder_list)
        # print('t=',end='')
        # print(t)
        num_batches = int(len(folder_list)/batch_size) # calculate the number of batches
        #print('num_batches=',num_batches)
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,input_shape[3])) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,no_of_classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
    #             # print('folder=' + str(folder))
    #             print('iterated=',folder + (batch*batch_size))
    #             print('path=',source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                # print('imgs=' + str(imgs[:2]))
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    img_file_path = source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]
                    # print('img_file_path=' + img_file_path)
                    image = np.load(img_file_path)#.astype(np.float32)
                    #print(image.shape)
                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - image[:,:,0].min())/(image[:,:,0].max() - image[:,:,0].min()) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - image[:,:,1].min())/(image[:,:,1].max() - image[:,:,1].min()) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - image[:,:,2].min())/(image[:,:,2].max() - image[:,:,2].min()) #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            #batch_labels = to_categorical(batch_labels, 5) #5 classes
            #batch_labels.shape
            #batch_labels = batch_labels.reshape(-1, 1)
            #print('yielding normal batch=>{}'.format(batch))
            #print('batch_data=>{}'.format(str(batch_data)))
            #print('batch_labels=>{}'.format(str(batch_labels)))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        # write the code for the remaining data points which are left after full batches
        remaining_size = len(folder_list) - (batch_size*num_batches)
        if (remaining_size > 0):
            #print('remaining_size=',remaining_size)
            batch_data = np.zeros((remaining_size,x,y,z,input_shape[3])) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((remaining_size,no_of_classes)) # batch_labels is the one hot representation of the output
            for folder in range(remaining_size):
                #print('new_folder=',folder)
                #print('new path=',source_path+'/'+ t[folder].split(';')[0])
                imgs = os.listdir(source_path+'/'+ t[folder].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    img_file_path = source_path+'/'+ t[folder].strip().split(';')[0]+'/'+imgs[item]
                    # print('img_file_path=' + img_file_path)
                    image = np.load(img_file_path)#.astype(np.float32)                                        
                    #print(image.shape)
                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - image[:,:,0].min())/(image[:,:,0].max() - image[:,:,0].min()) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - image[:,:,1].min())/(image[:,:,1].max() - image[:,:,1].min()) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - image[:,:,2].min())/(image[:,:,2].max() - image[:,:,2].min()) #normalise and feed in the image
                batch_labels[folder, int(t[folder].strip().split(';')[2])] = 1
            #batch_labels = to_categorical(batch_labels, 5) #5 classes
            #batch_labels = batch_labels.reshape(-1, 1)
            #print('yielding remaining batch=>',batch)
            #print(batch_labels)
            yield batch_data, batch_labels
            Model_Execution = True

# ## test code to test generator function
# train_path = f'{root_dir}/numpy/train'
# num_train_sequences = len(train_doc)
# train_generator = generator(source_path=train_path, folder_list=train_doc, batch_size=batch_size, ablation=False)
# index = 0
# for batch_data, batch_labels in train_generator: 
#     index += len(batch_data)    
#     print(batch_data.shape,batch_labels.shape) 

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [12]:
curr_dt_time = datetime.datetime.now()
train_path = f'{root_dir}/numpy/train'
val_path = f'{root_dir}/numpy/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout,LeakyReLU
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger,EarlyStopping
from keras import optimizers

In [0]:
#model.add(Conv3D(filters=32, kernel_size=(3,3,3), strides=(1, 1, 1), padding='same', activation='relu',data_format='channels_last'))
#model.add(MaxPooling3D(pool_size=(2, 2, 1)))
#model.add(Dense(1024))


#write your model here

##################### model 1 ################
# model = Sequential()
# model.add(Conv3D(filters=32, kernel_size=(3,3,3), strides=(1, 1, 1), activation=None, input_shape=(len(list(range(0,30,1))),120,160,3),data_format='channels_last'))
# model.add(LeakyReLU(alpha=0.1))
# model.add(MaxPooling3D(pool_size=(1, 1, 1)))
# model.add(Conv3D(filters=64, kernel_size=(3,3,3), strides=(1, 1, 1), activation=None))
# model.add(LeakyReLU(alpha=0.1))
# model.add(Flatten())
# model.add(Dense(5, activation='softmax')) # 5 are number of classes

In [0]:
# ###################### model 2 ################
# model = Sequential()
# model.add(Conv3D(64, kernel_size=(3,3,3), activation='relu', input_shape=input_shape,data_format='channels_last'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
# model.add(Conv3D(128, (3,3,3), activation='relu'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
# model.add(Conv3D(256, (3,3,3), activation='relu'))
# model.add(Conv3D(256, (3,3,3), activation='relu'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
# model.add(Conv3D(512, (3,3,3), activation='relu'))
# model.add(Conv3D(512, (3,3,3), activation='relu'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
# # model.add(Conv3D(512, (3,3,3), activation='relu'))
# # model.add(Conv3D(512, (3,3,3), activation='relu'))
# # model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
# model.add(Flatten())
# model.add(Dense(2048,activation='relu'))
# model.add(Dense(2048,activation='relu'))
# model.add(Dense(no_of_classes, activation='softmax'))

In [0]:
# ###################### model 2 ################
# model = Sequential()
# model.add(Conv3D(64, kernel_size=(3,3,3), activation='relu', input_shape=input_shape,data_format='channels_last'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
# model.add(Conv3D(128, (3,3,3), activation='relu'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
# model.add(Conv3D(256, (3,3,3), activation='relu'))
# model.add(Conv3D(256, (3,3,3), activation='relu'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
# model.add(Conv3D(512, (3,3,3), activation='relu'))
# model.add(Conv3D(512, (3,3,3), activation='relu'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
# # model.add(Conv3D(512, (3,3,3), activation='relu'))
# # model.add(Conv3D(512, (3,3,3), activation='relu'))
# # model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
# model.add(Flatten())
# model.add(Dense(2048,activation='relu'))
# model.add(Dense(2048,activation='relu'))
# model.add(Dense(no_of_classes, activation='softmax'))

In [17]:
###################### model 3 - deep model ################
model = Sequential()
model.add(Conv3D(64, kernel_size=(3,3,3), activation='relu', input_shape=input_shape,data_format='channels_last'))
model.add(Conv3D(64, (3,3,3), padding='same',activation='relu'))
model.add(MaxPooling3D(pool_size=(1,2,2), strides=(1,2,2)))
model.add(Conv3D(128, (3,3,3), padding='same',activation='relu'))
model.add(Conv3D(128, (3,3,3),  activation='relu'))
model.add(MaxPooling3D(pool_size=(1,2,2), strides=(1,2,2)))
model.add(Conv3D(256, (3,3,3), activation='relu'))
model.add(Conv3D(256, (3,3,3), activation='relu'))
model.add(Conv3D(256, (3,3,3),  activation='relu'))
model.add(Conv3D(256, (3,3,3),  activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model.add(Flatten())
model.add(Dense(2048,activation='relu'))
model.add(Dense(2048,activation='relu'))
model.add(Dense(no_of_classes, activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 13, 98, 98, 64)    5248      
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 13, 98, 98, 64)    110656    
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 13, 49, 49, 64)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 13, 49, 49, 128)   221312    
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 11, 47, 47, 128)   442496    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 11, 23, 23, 128)   0         
_________________________________________________________________
conv

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [18]:
optimiser = optimizers.Adam(lr=1e-4)#, decay=1e-6, momentum=0.9, nesterov=True) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 13, 98, 98, 64)    5248      
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 13, 98, 98, 64)    110656    
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 13, 49, 49, 64)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 13, 49, 49, 128)   221312    
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 11, 47, 47, 128)   442496    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 11, 23, 23, 128)   0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 9, 21, 21, 256)    884992    
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [0]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [0]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

#checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
my_model_checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=True, mode='auto', period=1)
csv_logger = CSVLogger('training.cnn3d.txt',separator=',', append=True)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1)
reduce_LR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-10,verbose=1) # write the REducelronplateau code here
callbacks_list = [reduce_LR,csv_logger,my_model_checkpoint,early_stopping] #checkpoint

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [0]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [22]:
#model.load_weights('./03-model_init_2019-06-1205_47_36.337812/model-00010-0.84542-0.67270-1.15618-0.59000.h5')
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
23/23 [==============================] - 147s 6s/step - loss: 1.6147 - categorical_accuracy: 0.2012 - val_loss: 1.6092 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_2019-06-1208_44_45.080481/model-00001-1.61486-0.19608-1.60921-0.20000.h5
Epoch 2/50
23/23 [==============================] - 114s 5s/step - loss: 1.6083 - categorical_accuracy: 0.2114 - val_loss: 1.6053 - val_categorical_accuracy: 0.1600

Epoch 00002: saving model to model_init_2019-06-1208_44_45.080481/model-00002-1.60861-0.20664-1.60529-0.16000.h5
Epoch 3/50
23/23 [==============================] - 114s 5s/step - loss: 1.5944 - categorical_accuracy: 0.2215 - val_loss: 1.5387 - val_categorical_accuracy: 0.2800

Epoch 00003: saving model to model_init_2019-06-1208_44_45.080481/model-00003-1.59379-0.21719-1.53874-0.28000.h5
Epoch 4/50
23/23 [==============================] - 114s 5s/step - loss: 1.4536 - categorical_accuracy: 0.3836 - va

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 13, 98, 98, 64)    5248      
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 13, 98, 98, 64)    110656    
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 13, 49, 49, 64)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 13, 49, 49, 128)   221312    
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 11, 47, 47, 128)   442496    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 11, 23, 23, 128)   0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 9, 21, 21, 256)    884992    
__________

In [0]:
os.rename('/content/cloned-repo/Project_data/model_init_2019-06-1208_44_45.080481','/content/cloned-repo/Project_data/04-model_init_2019-06-1208_44_45.080481')


In [0]:
model.save(model_name + 'COMPLETE_model-00020-0.27388-0.88688-1.37700-0.57000.h5')

In [31]:
os.makedirs('/content/gdrive',exist_ok=True)
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [36]:
import shutil
shutil.copytree('/content/cloned-repo/Project_data/04-model_init_2019-06-1208_44_45.080481', '/content/gdrive/My Drive/gesture_recognition/04-model_init_2019-06-1208_44_45.080481')

'/content/gdrive/My Drive/gesture_recognition/04-model_init_2019-06-1208_44_45.080481'